질문과 answer 두가지 입력을 받는 함수형 api 모델 제작

In [ ]:
from keras.models import Model
from keras import layers
from keras import Input

text_size = 10000
question_size = 10000
answer_size = 500

text_input = Input(shape = (None , ), dtype = 'int32', name = 'text')
embedded_text = layers.Embedding(text_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis = -1)

answer = layers.Dense(answer_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['acc'])


In [ ]:
model.summary()

#데이터주입


In [ ]:
import numpy as np
import tensorflow
num_sample = 1000
max_length = 100

text = np.random.randint(1, text_size, size = (num_sample, max_length))
question = np.random.randint(1, question_size, size = (num_sample, max_length))
answers = np.random.randint(0, answer_size, size = num_sample)
answers = tensorflow.keras.utils.to_categorical(answers)

model.fit([text,question], answers, epochs = 10, batch_size=128)

#출력

#7-7 텐서보드사용

In [ ]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)


In [ ]:
model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name = 'embedding'))
model.add(layers.Conv1D(32,7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32,7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.summary()


In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
%mkdir my_log_dir

In [ ]:
callbacks = [
             keras.callbacks.TensorBoard(
                 log_dir = 'my_log_dir',
                 histogram_freq = 1,
                 embeddings_freq = 1,
             )
]

history = model.fit(
    x_train, y_train,
    epochs = 20,
    batch_size = 128,
    validation_split = 0.2,
    callbacks = callbacks
)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir = my_log_dir

In [ ]:
%tensorboard --logdir my_log_dir

In [ ]:
!kill 173